# Introduction

In [84]:
# The idea of this competition, is to predict sales for mnay product families for stores in ecuador.
#Train and test share: date, store_nbr, family(product_family), onpromotion (number of products of the family that were on promotion)
#Train also has the sales.
#Test is what we are trying to predict in the competition

#Stores contains store metadata, we might want to join that data into our train and test tables to improve predictions if they are not redundant with the store_nbr (aka if at least some of the stores share their values).
 #stores metadata
#type and cluster are not redundant. see stores_df[['type', 'cluster']].drop_duplicates()#.pivot(columns='cluster')

#Daily oil price, keep in mind oil price affects ecuador economical health.
#We probably want to join it into our dataframe, but keep in mind it seems to have some null values. We probably want to fill downwards so that it takes its value from the previous date but well have to see. We could also average between the next and previous day.


#While its not clear it seems to be the total transactions of the store on that day


#A table containing holiday events. 
#its worth noting that some events were transfered in date so we need to take into account hte actual date. transfered column True means that the holiday wasnt actually celebrated that date.
#Also, keep in mind that some seem to be regional and some local. This could mean that we need to take into account the region in considering them.
#Some holidays were of type bridge, meaning  they are extra days added to the same holiday.
#This are often compensated by making some not working days(ie saturday) working days. This are of type workday.
#holiday_type additional means that it isnt an actual holiday, but actually the extension of one.

#Additonally to these tables wages on the public sector are paid on the 15th and last day of each month which could affect supermarket sales
#A great earthquiake struck ecuador on april 16 2016 which resulted in donations possibly affecting sales.


#TLDR we need to add many features to the train and test df.
#First lets make some column names easier to understand. Specially after joining.

In [1]:
import os
import json
with open('./.kaggle/kaggle.json') as credentials_file:
    credentials_dict = json.load(credentials_file)
    os.environ['KAGGLE_USERNAME'] = credentials_dict['username']
    os.environ['KAGGLE_KEY'] = credentials_dict['key']
    import kaggle
from zipfile import ZipFile
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm

In [49]:
def download_dataset():
    kaggle.api.competition_download_files(competition='store-sales-time-series-forecasting', path='./dataset', force=False, quiet=True)
    with ZipFile('./dataset/store-sales-time-series-forecasting.zip') as dataset_zip:
        dataset_zip.extractall('./dataset')

def get_raw_dfs() ->tuple:
    """Get dataframes containing all the info in the dataset."""
    train_df_inc = pd.read_csv('./dataset/train.csv', index_col='id')
    test_df_inc = pd.read_csv('./dataset/test.csv', index_col='id')
    stores_df = pd.read_csv('./dataset/stores.csv', index_col='store_nbr')
    oil_df = pd.read_csv('./dataset/oil.csv') 
    transactions_df = pd.read_csv('./dataset/transactions.csv')
    holiday_events_df = pd.read_csv('./dataset/holidays_events.csv')
    sample_submission_df = pd.read_csv('./dataset/sample_submission.csv')
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df

def rename_raw_dfs_cols(train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df) ->tuple:
    """Rename the columns of the raw dataframes so they are more easily understandable"""
    train_df_inc = train_df_inc.rename(columns={'family':'product_family', 'onpromotion':'products_of_family_on_promotion'})
    test_df_inc = test_df_inc.rename(columns={'family':'product_family'})
    oil_df = oil_df.rename(columns={'dcoilwtico':'oil_price'})
    stores_df = stores_df.rename(columns={'transactions':'all_products_transactions', 'type':'store_type', 'cluster':'store_cluster', 'city':'store_city', 'state':'store_state'})
    holiday_events_df = holiday_events_df.rename(columns={'type':'day_type', 'locale':'holiday_locale_type', 'locale_name':'holiday_locale','description':'holiday_reason', 'transferred':'holiday_transferred'})
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df

def complete_features_dataset(features_df:pd.DataFrame, stores_df:pd.DataFrame, oil_df:pd.DataFrame, transactions_df:pd.DataFrame, holiday_events_df:pd.DataFrame)->pd.DataFrame:
    """Add relevant columns from the auxiliary dataframes into a features dataset, be it the train set or the test set."""
    full_features_df = features_df.merge(oil_df, on='date',how='left')
    full_features_df = full_features_df.merge(stores_df, on=['store_nbr'], how='left') #Need to also consider date. 'date', nvm
    full_features_df = full_features_df.merge(holiday_events_df, on='date')
    full_features_df = full_features_df.merge(transactions_df, on=['date', 'store_nbr'])
    return full_features_df

def reorder_features_dataset(features_df):
    """Reorder the columns in the feature dataframe so the table becomes easier to understand and inspect. Does not affect the rows."""
    return features_df # TODO Order to columns more logically

def one_hot_encode_features_dataset(features_df):
    """One hot encodes the columns, using their name as prefix, and adding them into the same place the original was, while removing the original """
    names_of_columns_to_ohe= [ 'store_nbr', 'product_family', 'store_city','store_state',
                      'store_type', 'day_type', 'holiday_locale_type', 'holiday_locale',
                       'holiday_description', 'holiday_transferred']
                    #holiday_transferred may be better vectorized. Day type might be too. holiday_locale_type too.
                    #This is because their values might have meaning in the order.
    
    for name_of_column_to_ohe in names_of_columns_to_ohe:
        index_of_col_to_ohe = features_df.columns.get_loc(name_of_column_to_ohe)
        one_hot_encoded_column:pd.DataFrame = pd.get_dummies(features_df[name_of_column_to_ohe], dummy_na=True, prefix=name_of_column_to_ohe)
        
        # Split the dataframe into two parts: before and after the position of the original column
        columns_before_col_to_ohe = features_df.iloc[:, :index_of_col_to_ohe]
        columns_after_col_to_ohe = features_df.iloc[:, index_of_col_to_ohe+1:] #Exclude the current column

        # Concatenate the two parts with the one-hot encoded dataframe in between
        features_df = pd.concat([columns_before_col_to_ohe, one_hot_encoded_column, columns_after_col_to_ohe], axis=1)

    return features_df
    #TODO #return the dataset and the references

def normalize_numerical_features(features_df):
    """ """
    return features_df
    #TODO #return the dataset and the normalizer

def fill_missing_values(features_df):
    """ """

def prepare_features_df(inc_dataframe:pd.DataFrame, stores_df, oil_df, transactions_df, holiday_events_df):
    """Call all the preprocessing methods in order, and prepare a features dataframe for deep learning, be it the train set or the test set"""
    #TODO READ and rename columns of the dataset, and then process them here.
    complete_features_df = complete_features_dataset(inc_dataframe, stores_df, oil_df, transactions_df, holiday_events_df)

    #Usar locale como detail instead? No. Si necesito both locale and detail ejemplo terremoto va a ser problematico. 
    #Aniadir una columna, holiday_reason_details y despues modificarla junto a holiday description (que en realidad ya no se llama asi sino holiday_reason)
    holiday_descriptions = complete_features_df['holiday_description']
    
    holiday_descriptions = complete_features_df['holiday_description'].drop_duplicates()
    for description in holiday_descriptions.sort_values():
        print(description)
    
    
    ordered_features_df = reorder_features_dataset(complete_features_df)
    ohe_features_df = one_hot_encode_features_dataset(ordered_features_df)
    return ohe_features_df

def window_dataset():
    """Window the dataset so that it can be used for training a neural network."""
    #Aka create a window of days with values and then the unknown value for the next day. and then slide that window forward to create another data point.
    #TODO
    return

def remove_seasonality():
    #perform fourier transform? Include the relevant data for example weekday as datapoint? Make sure to add day of year, day of month, day of week as features.
    #TODO?
    return

def train_val_split_dataset():
    """Split the datataset so that it can be used for training and then validation"""
    #TODO, probably the most sensible approach for a simple time series analysis is to use the last data for validation,
    #though i can imagine other approaches which could lead to better peformance, for example masking like bert does for text data.
    #Or creating crossval sets by skipping part of the time before prediction
    return

def prepare_datasets():
    train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df = rename_raw_dfs_cols(*get_raw_dfs())
    train_y_truth = train_df_inc.pop('sales')
    train_df, test_df = prepare_features_df(train_df_inc, stores_df, oil_df, transactions_df, holiday_events_df), prepare_features_df(test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df)
    return train_df, test_df, train_y_truth, sample_submission_df


train_df, test_df, train_y_truth, sample_dataset = prepare_datasets()

KeyError: 'holiday_description'

In [46]:
pd.set_option('display.max_rows', 1000)

In [48]:
train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df = rename_raw_dfs_cols(*get_raw_dfs())
holiday_events_df.drop_duplicates()
#.drop(columns=['date'])

,date,day_type,holiday_locale_type,holiday_locale,holiday_description,holiday_transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False
